


Cross Validation & Naive Bayes Lab - SMS Spam Classification
===============
* orignally developed by Ankit Jain
* modified by Justin Breucop
* modified by Dylan Hercher

Data source: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

# Section 1)

## Cross Validation from Scratch

Let's build it the function together! The steps to cross validation are:
1. Randomly separate your training set into _k_ groups
2. For each group _k_:
>1. Train your model on the other groups
>2. Score your model using group _k_ as validation
>3. Save your score and move to your next group

3. Add your _k_ scores and divide by _k_ to get your average score

In [16]:
# Importing Packages 
import numpy as np
import pandas as pd
import random

from sklearn.neighbors import KNeighborsClassifier

In [17]:
model = KNeighborsClassifier(10)

df = pd.DataFrame.from_csv('../data/titanic-train.csv',index_col=None)[['Age','Pclass','SibSp','Survived']].dropna()
to_predict = "Survived"
features=['Age','Pclass','SibSp']
data = df[features]
label = df[to_predict]
folds=5

In [6]:
data.head()

,Age,Pclass,SibSp
0,22,3,1
1,38,1,1
2,26,3,0
3,35,1,1
4,35,3,0


In [7]:
label.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

Let's build in class!

In [8]:
#Function here:
def cross_validate_df(data,label,model,k):
    """Return Average score across k iterations
    Parameters
    -----------
    data : DataFrame with features to use in X
    label : Series with target y
    model : ML Model to use
    k : int number of iterations
    """
    # TODO: Design the cross validation function
    # it may be helpful to try writing the function with k=1 first
    
    return None

In [49]:
train_slice = []
test_slice = []
positions = data.index.values

k=5

np.random.shuffle(positions)

for i in range(k):
     
    section_start = i*len(data)/k
    section_end = (i+1)*len(data)/k
    test_slice.append(positions[ section_start:section_end ])
    
    train_p1 = positions[ :section_start ]
    train_p2 = positions[ section_end: ]
    train_combo = np.concatenate([train_p1,train_p2])
    train_slice.append(train_combo)
    
scores = 0
    
for train_sl, test_sl in zip(train_slice,test_slice):

    model.fit(data.loc[train_sl],label.loc[train_sl])
    k_score = model.score(data.loc[test_sl],label.loc[test_sl])
    scores += k_score
    print k_score
        
return scores/k
        

    
 
    
        

0.647887323944
0.72027972028
0.713286713287
0.685314685315
0.692307692308


SyntaxError: 'return' outside function (<ipython-input-49-2375de0aa861>, line 29)

In [50]:
# Once Complete test your function here
cross_validate_df(data,label,model,5)

### Built-in Cross Validation
There is also a very simple cross validation function provided by sklearn

In [37]:
from sklearn.cross_validation import cross_val_score

# Results of using the built-in cross validation
# Note: Default is Kfold, but within sklearn.cross_validation
# there are many types of validation that can be used
cross_val_score(model, data, label, cv=3)

array([ 0.64335664,  0.69230769,  0.6993007 ,  0.74125874,  0.67605634])

In [39]:
cross_val_score(model, data, label, cv=3)

array([ 0.64335664,  0.69230769,  0.6993007 ,  0.74125874,  0.67605634])

In [38]:
np.mean(cross_val_score(model, data, label, cv=3))

0.67794371954950439

# Section 2)
## Naive Bayes and SMS Spam Classification

In [51]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


In [52]:
## READING IN THE DATA
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)

In [53]:
# examine the data
df.head()

,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [54]:
df[df.label=='spam'].head()

,label,msg
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [55]:
df.label.value_counts()

ham     4825
spam     747
dtype: int64

In [ ]:
df.msg.describe()

In [ ]:
# Convert the label into a binary variable
# Remember the map function we learned before?
df['label'] = df.label.map({'ham': 0 , 'spam':1})

In [ ]:
df.head()

In [ ]:
# split into training and testing sets by calling sklearn lib
# by default, the data set is split into 0.75 (training) and 0.25 (testing)
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

In [ ]:
print X_train.shape
print X_train

In [ ]:
X_test.shape

Now we need to convert the text into feature vectors which can be used for machine learning purposes.
We will use the scikit function of CountVectorizer to 'convert text into a matrix of token counts'

 http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

#### Lets try a simple example

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# start with a simple example
train_simple = ['call you tonight',
                'Call me a cab',
                'please call me... PLEASE!']

In [ ]:
# learn the 'vocabulary' of the training data
vect = CountVectorizer(decode_error='ignore')
vect.fit(train_simple)
vect.get_feature_names()

In [ ]:
# transform training data into a 'document-term matrix'
train_simple_dtm = vect.transform(train_simple)
train_simple_dtm.toarray()

In [ ]:
# We can see how we've adjusted our data easily!
# examine the vocabulary and document-term matrix together
print train_simple
    
pd.DataFrame(train_simple_dtm.toarray(), columns=vect.get_feature_names())

In [ ]:
# transform testing data into a document-term matrix (using existing vocabulary)
test_simple = ["please don't call me"]
test_simple_dtm = vect.transform(test_simple)
test_simple_dtm.toarray()

pd.DataFrame(test_simple_dtm.toarray(), columns=vect.get_feature_names())

#### Question:  How does the above test_simple show how things can go wrong?

#### Exercise: Using the dataset below
   * Vectorize the text
   * Store the results in a DataFrame
   * Show word counts (hint: one dataframe describer can do this)
   * Transform the test text

In [ ]:
train_exp = ['where is my taco?',
                'did I eat the taco',
                'I can easily eat my way through that whole box of tacos!',
                'I think way too much about tacos, huh',
                'taco, taco, taco!!!'                
               ]
test_exp = [
    'where did he go?', 'how long did the whole thing last', 'lets go eat one taco or multiple tacos'
]

## Vectorizing our SMS Dataset

In [ ]:
# instantiate the vectorizer ( use variable name as vect)
vect = CountVectorizer(decode_error = 'ignore')
vect.fit(X_train)
vect.get_feature_names()

In [ ]:
# transform testing data into a document-term matrix: Use Variable name as test_dtm
train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)
print test_dtm

In [ ]:
# Get the length and names of the feature names
train_features = vect.get_feature_names()
len(train_features)

In [ ]:
train_features[:50]

In [ ]:
train_features[-50:]

In [ ]:
# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

In [ ]:

# Revisit Numpy
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print arr[0, 0]
print arr[1, 3]
print arr[0, :]
print arr[:, 0]
print np.sum(arr)
print np.sum(arr,axis = 0)
print np.sum(arr,axis = 1)




In [ ]:
# exercise: calculate the number of tokens in the 0th message in train_arr
print np.sum(train_arr[0,:])

In [ ]:

# exercise: count how many times the 0th token appears across ALL messages in train_arr
print np.sum(train_arr[:,0])

In [ ]:
# exercise: count how many times EACH token appears across ALL messages in train_arr
print np.sum(train_arr, axis=0)

In [ ]:
# exercise: create a DataFrame of tokens with their counts.


### Let's build the model with Naive Bayes Now

http://scikit-learn.org/stable/modules/naive_bayes.html

In [ ]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(train_dtm, y_train)

In [ ]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm)
preds

In [ ]:
# compare predictions to true labels
from sklearn import metrics
print metrics.accuracy_score(y_test, preds)
print metrics.confusion_matrix(y_test, preds)

# confusion matrix: http://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
# exercise: show the message text for the false positives
X_test[(y_test == 0) & (preds == 1)]

In [ ]:
# exercise: show the message text for the false negatives
X_test[y_test > preds]
# or
X_test[(y_test == 1) & (preds == 0)]

In [ ]:
## USING ALL DATA AND CROSS-VALIDATION, run NB again


In [ ]:
# Here is the condensed code needed for reference
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)
X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

vect = CountVectorizer(decode_error = 'ignore')

vect.fit(X_train)
vect.get_feature_names()

train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)

In [ ]:
## EXERCISE: CALCULATE THE 'SPAMMINESS' OF EACH TOKEN

# create separate DataFrames for ham and spam ( df_ham and df_spam)


In [ ]:
# learn the vocabulary of ALL messages and save it


In [ ]:
# create document-term matrix of ham, then convert to a regular array


In [ ]:
# create document-term matrix of spam, then convert to a regular array


In [ ]:
# count how many times EACH token appears across ALL messages in ham_arr


In [ ]:
# count how many times EACH token appears across ALL messages in spam_arr


In [ ]:
# create a DataFrame of tokens with their separate ham and spam counts


In [ ]:
# add one to ham counts and spam counts so that ratio calculations (below) make more sensse


In [ ]:
# calculate ratio of spam-to-ham for each token


In [ ]:
# advanced: implement your own naive bayes classifier
